***
#### About this document
This is the assignment for the UCL CASA module Data Science for Spatial Systems (CASA0006) <br>
**Code is hosted [here]**<br>
**Data is availble [here]**
***

# Nonlinear effects of public transport accessibility on urban development: 
<p style="color:DimGray; text-align:left; font-size:20px"><i> the case of Chongqing City </i></p>
<a id="ID_title"></a>

<p style="color:DimGray; text-align:left;"><i>This paper explores data from . <br>It highlights</i></p>

---

<a id="ID_intr"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## 1.0 | Introduction

data [transport](https://restapi.amap.com/v3/bus/linename?s=rsv3&extensions=all&key=d39b1ec75e5f1f9033b1bc00952c4ced&output=json&city=重庆&offset=2&keywords=轨道交通1号线&platform=JS) json file

In [33]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [28]:
import requests
import os
import json
import math
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from glob import glob
#!pip install ChangeCoordinate
from ChangeCoordinate import ChangeCoord
#!pip install folium mapclassify


<a id="ID_lit"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore]("#section_ID") | [6.0 Analysis](#ID_Intro) | [7.0 Results]("#section_ID") | [8.0 Discussion](#ID_Intro) | [Bibliography](#ID_Intro) | [Appendix](#ID_Intro) |

## 2.0 | Literature review

<a id="ID_method"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## 3.0 | Methodology

<a id="ID_data"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## 4.0 | Datasets


In [2]:
def get_data(line):
    print(line)
    url=("https://restapi.amap.com/v3/bus/linename?s=rsv3&extensions=all&key=6e0e286fb0ddd87779b1ff5ee653e9fa&output=json&city=重庆&offset=2&keywords={}&platform=JS&jscode=88de710fc1d55dbefd60d24831c27d5f"). \
    format(line)
    
    r=requests.get(url).json()
    df = pd.DataFrame(r['buslines'][0]['busstops'])
    return df
# spare url=("https://restapi.amap.com/v3/bus/linename?s=rsv3&extensions=all&key=d39b1ec75e5f1f9033b1bc00952c4ced&output=json&city=重庆&offset=2&keywords={}&platform=JS"). \
# spare key: 6e0e286fb0ddd87779b1ff5ee653e9fa

In [3]:
line_names = ['轨道交通1号线', '轨道交通2号线', '轨道交通3号线', '轨道交通3号线北延线','轨道交通4号线', '5号线', 
              '轨道交通6号线','轨道交通6号线支线（国博线）','轨道交通9号线','轨道交通10号线',
              '轨道交通江跳线','轨道交通环线（外环）','轨道交通环线（内环）']

In [4]:
dx = pd.concat([get_data(line) for line in line_names], axis=0)

轨道交通1号线
轨道交通2号线
轨道交通3号线
轨道交通3号线北延线
轨道交通4号线
5号线
轨道交通6号线
轨道交通6号线支线（国博线）
轨道交通9号线
轨道交通10号线
轨道交通江跳线
轨道交通环线（外环）
轨道交通环线（内环）


In [5]:
dx

,id,location,name,sequence
0,BV10049208,"106.585845,29.564142",朝天门,1
1,BV10045951,"106.583159,29.559168",小什字,2
2,BV10045314,"106.571829,29.553173",较场口,3
3,BV10045298,"106.56291,29.554914",七星岗,4
4,BV10045318,"106.548514,29.552864",两路口,5
...,...,...,...,...
29,BV11102222,"106.453159,29.495547",华龙,30
30,BV10668313,"106.439385,29.500105",重庆西站,31
31,BV11102218,"106.442419,29.517693",上桥,32
32,BV11102223,"106.449365,29.524457",凤鸣山,33


In [6]:
ccg = ChangeCoord()

x = dx.iloc[0]['location']
lng, lat = float(x.split(',')[0]), float(x.split(',')[1])
ccg.gcj02_to_wgs84(lng,lat)

[106.58215932367729, 29.567034062350533]

In [7]:
dx['coord_wg84'] = dx['location'].apply(lambda x: ccg.gcj02_to_wgs84(float(x.split(',')[0]), 
                                                                     float(x.split(',')[1])))

In [16]:
dx['coord_wg84'].str.get(1)
geometry = gpd.points_from_xy(dx['coord_wg84'].str.get(0), 
                              dx['coord_wg84'].str.get(1))

metro_sts = gpd.GeoDataFrame(dx[['id', 'location', 'name', 'sequence']],
                                geometry=geometry,
                                crs = 'epsg:4326')
metro_sts.to_file('data/cq_metro_sts.geojson', driver='GeoJSON')
metro_sts.to_file('data/cq_metro_sts.shp')

In [9]:
metro_sts.explore()

In [ ]:
def get_amap_poi(city, keywords, page):
    amap_api_url = "https://restapi.amap.com/v3/place/text"
    # replace with your web api key
    api_key = "8dd06dd6bddfefffdc9e169b00d5191b"

    params = {"key": api_key,"keywords": keywords,"offset": 1000,"page": page,"city": city,"output": "json"}

    response = requests.get(amap_api_url, params=params)
    result = response.json()

    if result["status"] == "1" and int(result["count"]) > 0:
        pois = result["pois"]
        return pois
    else:
        print("not found pois or error")
        return []

def main():
    city_name = "重庆市" 
    keyword = "餐饮服务"
    page_num = 1

    pois = get_amap_poi(city_name, keyword, page_num)
    output_label = ['name', 'location', 'tel', 'pname', 'cityname', 'adname', 'address']

    if pois:
        for poi in pois:
            info = []
            for label in output_label:
                info.append(poi[label])
            print(info)
    else:
        print("not found pois or error")

if __name__ == "__main__":
    main()


In [29]:
def get_amap_poi(city, keywords, page):
    amap_api_url = "https://restapi.amap.com/v3/place/text"
    # replace with your web api key
    api_key = "8dd06dd6bddfefffdc9e169b00d5191b"

    params = {"key": api_key,"keywords": keywords,"offset": 1000,"page": page,"city": city,"output": "json"}

    response = requests.get(amap_api_url, params=params)
    result = response.json()

    if result["status"] == "1" and int(result["count"]) > 0:
        pois = result["pois"]
        return pois
    else:
        print(f"Not found pois or error for keyword: {keywords}")
        return []

def save_to_csv(data, filename):
    file_path = os.path.join("data", filename)
    with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['name', 'location', 'tel', 'pname', 'cityname', 'adname', 'address'])
        writer.writerows(data)

def main():
    city_name = "重庆市" 
    keywords_list = [] 
    page_num = 1
    output_label = ['name', 'location', 'tel', 'pname', 'cityname', 'adname', 'address']

    for keyword in keywords_list:
        pois = get_amap_poi(city_name, keyword, page_num)
        
        if pois:
            output_data = []
            for poi in pois:
                info = [poi[label] for label in output_label]
                output_data.append(info)
            
            save_to_csv(output_data, f"{keyword}.csv")
            print(f"Data saved to data/{keyword}.csv")
        else:
            print(f"Not found pois or error for keyword: {keyword}")

if __name__ == "__main__":
    main()


Data saved to data/餐饮服务.csv
Data saved to data/购物服务.csv


In [22]:
metro_sts = gpd.read_file('data/cq_metro_sts.geojson')
metro_sts.head(2)
metro_sts.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   id        326 non-null    object  
 1   location  326 non-null    object  
 2   name      326 non-null    object  
 3   sequence  326 non-null    object  
 4   geometry  326 non-null    geometry
dtypes: geometry(1), object(4)
memory usage: 12.9+ KB


<a id="ID_explore"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## 5.0 | Data preparation

In [31]:
metro_sts = metro_sts.to_crs(3857)
metro_sts['geometry'] = metro_sts['geometry'].buffer(600)

In [ ]:
poi = poi.to_crs(3857)

In [ ]:
poi_within_600 = gpd.sjoin(poi, metro_sts, op='within')
poi_within_600.rename(columns={'name_left': 'MSA'}, inplace=True)

In [ ]:
poi_within_600.groupby('MSA')['name'].count().reset_index()

<a id="ID_analyse"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## 6.0 | Analysis

<a id="ID_conclude"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## 7.0 | Discussion and conclusion

<a id="ID_biblio"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |


***
## Bibliography


<a id="ID_append"></a>
| [1.0 Intro](#ID_intr) | [2.0 Lit. review](#ID_lit) | [3.0 Method](#ID_method) | [4.0 Data](#ID_data) | [5.0 Explore](#ID_explore) | [6.0 Analysis](#ID_analyse) | [7.0 Discussion and conclusion](#ID_conclude) | [Bibliography](#ID_biblio) | [Appendix](#ID_append) |

## Appendix